In [ ]:
"Hi"

In [ ]:
# %pip install langgraph langchain-openai pydantic python-dotenv

1. Import libraries

In [ ]:
from langgraph.graph import StateGraph, END
from pydantic import BaseModel
import json
import re
import random
from typing import List

2. Define the Agent State

In [ ]:
class CICDState(BaseModel):
    logs: str = ""
    diagnosis: str = ""
    fix_patch: str = ""
    pr_state: str = ""   # created / approved / declined

### This agent will: Diagnosis Agent – Reads Logs & Detects Issue

Scan CI/CD logs text

Detect common Azure/Terraform/YAML/Python issues

Produce a structured diagnosis

In [ ]:
def diagnosis_agent(state: CICDState):
    logs = state.logs.lower()
    detected = []

    # Detect common issues
    if "function app name" in logs and "too long" in logs:
        detected.append("FunctionAppNameTooLong")

    if "terraform" in logs and "resource already exists" in logs:
        detected.append("TerraformDuplicateResource")

    if "yaml" in logs and "indentation" in logs:
        detected.append("YAMLIndentationError")

    if "module not found" in logs:
        detected.append("PythonModuleNotFound")

    if not detected:
        detected.append("UnknownError")

    diagnosis = f"Detected Issues: {detected}"
    print("🩺 Diagnosis Agent Output:", diagnosis)
    return {"diagnosis": diagnosis}

### Fixing Agent – Generates Patch
This agent reads the diagnosis and creates a code fix.

In [ ]:
def fixing_agent(state: CICDState):
    diagnosis = state.diagnosis

    patch = ""

    if "FunctionAppNameTooLong" in diagnosis:
        patch += "// Fix: Reduced function app name to <= 24 characters\n"
        patch += 'name = "my-func-app"\n'

    if "TerraformDuplicateResource" in diagnosis:
        patch += "// Fix: Renamed duplicate Terraform resource\n"
        patch += 'resource "azurerm_storage_account" "storage2" {}\n'

    if "YAMLIndentationError" in diagnosis:
        patch += "# Fix: Corrected YAML indentation\n"
        patch += "steps:\n  - task: Build\n    inputs:\n      script: python main.py\n"

    if "PythonModuleNotFound" in diagnosis:
        patch += "# Fix: Added missing dependency to requirements.txt\n"
        patch += "numpy==1.26.0\n"

    if "UnknownError" in diagnosis:
        patch += "// No known fix. Needs deeper inspection.\n"

    print("\n🛠 Fixing Agent Patch Generated:\n", patch)
    return {"fix_patch": patch}


### Pull request generator

In [ ]:
def create_pull_request(state: CICDState):
    print("\n📌 Creating Pull Request...")
    print("PR Diff:\n", state.fix_patch)
    state.pr_state = "created"
    return {"pr_state": "created"}


### Human Reviewer

In [ ]:
def human_review(state: CICDState):
    print("\n🔍 Human Review Needed")
    print("Patch:\n", state.fix_patch)

    decision = input("Approve PR? (yes/no): ").strip().lower()

    if decision == "yes":
        print("✅ PR Approved")
        return {"pr_state": "approved"}
    else:
        print("❌ PR Declined — triggering retry flow")
        return {"pr_state": "declined"}


### retry logic if pr rejected

In [ ]:
def retry_fix(state: CICDState):
    print("\n♻️ Retrying with enhanced reasoning...")
    new_patch = state.fix_patch + "\n# Retry patch → Improved auto-fix"
    print("New Patch:\n", new_patch)
    return {"fix_patch": new_patch, "pr_state": "created"}


### Build langgraph flow

In [ ]:
workflow = StateGraph(CICDState)

workflow.add_node("diagnosis", diagnosis_agent)
workflow.add_node("fix", fixing_agent)
workflow.add_node("pr", create_pull_request)
workflow.add_node("review", human_review)
workflow.add_node("retry", retry_fix)

workflow.set_entry_point("diagnosis")

workflow.add_edge("diagnosis", "fix")
workflow.add_edge("fix", "pr")
workflow.add_edge("pr", "review")

# PR declined → retry
workflow.add_conditional_edges(
    "review",
    lambda state: state.pr_state,
    {
        "approved": END,
        "declined": "retry"
    }
)

workflow.add_edge("retry", "pr")

app = workflow.compile()


In [ ]:
logs = """[Error] Function app name too long: must be <= 24 characters
Terraform failed: resource already exists
yaml indentation error at line 14"""


### Run the pipeline

In [ ]:
# with open("cicd_error.txt", "r") as f:
#     logs = f.read()

result = app.invoke({"logs": logs})
